In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection


In [3]:
%cd models/research
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 KB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 2

In [4]:
!python object_detection/builders/model_builder_tf2_test.py

2023-04-07 19:13:25.112928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 19:13:26.619420: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-04-07 19:13:35.343248: I tensorflow/compile

In [2]:
#importing necessary libraries
import glob
import cv2
import time
import numpy as np
import os
import imageio
from imutils.video import FPS
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util


In [3]:
#load the pretrained model
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/exportmodel_waymo_v120_efficientdet_d4/saved_model')
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

Elapsed time: 49.69901394844055s


In [4]:
#load the label map for the dataset
label_map_path='/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/waymo_labelmap.txt'
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [5]:
%cd /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection


In [ ]:
!git clone https://github.com/VolhaAnishchanka/diploma.git

Cloning into 'diploma'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 80 (delta 31), reused 58 (delta 18), pack-reused 0
Unpacking objects: 100% (80/80), 31.98 KiB | 23.00 KiB/s, done.


In [6]:
%cd /content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/diploma/tracking

/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/diploma/tracking


In [7]:
#to resolve the numpy compiled against different versions error when you import Tracker
!pip install --upgrade numpy
!pip install --upgrade tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.

In [8]:
from utils.tracker import Tracker
import cv2
import datetime
import argparse
import six

In [9]:
#testing on videos
directory = '/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/data/data/rearCamCalibrated/'

# initialize tracker
non_active_threshold = 15
active_threshold = 8
tracker = Tracker(active_threshold=active_threshold, non_active_threshold=non_active_threshold)

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        output_video_path = f
        #open the video
        cap = cv2.VideoCapture(output_video_path)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        fps = FPS().start()

        #set up the object for writing the created output video
        res_path = '/content/drive/MyDrive/SpecialProblems/VehicleProximity/waymoObjectDetection/results/'
        out = cv2.VideoWriter(res_path+filename, cv2.VideoWriter_fourcc('M','P','4','V'), 30, (frame_width,frame_height))

        currentframe = 0
        while (cap.isOpened() and currentframe<2000):
            ret, image_np = cap.read()
            if len((np.array(image_np)).shape) == 0:
                break
            path=res_path+str(currentframe)+'.jpg'
            #cv2.imwrite(path,image_np)
            input_tensor = np.expand_dims(image_np, 0)

            #get the detections
            detections = detect_fn(input_tensor)        
            label_id_offset = 1

            #create a copy of the input image to write to the output
            image_np_with_detections = image_np.copy()

            #writing the detections onto the image
            #first we need to get the boxes that are actually used
            boxes = detections['detection_boxes'][0].numpy()
            # get all boxes from an array
            max_boxes_to_draw = boxes.shape[0]
            # get scores to get a threshold
            scores = detections['detection_scores'][0].numpy()
            min_score_thresh=.5
            output_boxes=[]
            output_scores=[]
            output_classes=[]
            #img_pil=Image.fromarray(np.uint8(image_np)).convert('RGB')
            #draw = ImageDraw.Draw(img_pil)
            # iterate over all objects found
            for i in range(min(max_boxes_to_draw, boxes.shape[0])):                
                if scores[i] > min_score_thresh:
                    # boxes[i] is the box which will be drawn                
                    #im_width, im_height = image_np_with_detections.shape[:2]
                    ymin, xmin, ymax, xmax = boxes[i]
                    left = xmin * frame_width
                    top = ymin * frame_height
                    right = xmax * frame_width
                    bottom = ymax * frame_height
                    output_boxes.append((int(left),int(top),int(right),int(bottom)))
                    output_classes.append(detections['detection_classes'][0][i].numpy().astype(np.int32))
                    output_scores.append(scores[i])
            class_names=np.asarray([category_index[output_classes[i]]['name'] if output_classes[i] in six.viewkeys(category_index) else 'invalid class' for i in range(np.asarray(output_classes).shape[0]) ])
            objects = tracker.update(output_boxes)

            # update tracks
            for i in range(len(output_boxes)):
                
                #TODO: change color definition based on dist, convert bbox from xywh to xyxy, get class name of the tracked object, display dist info as well on bbox
                bbox = output_boxes[i]
                class_name = str(class_names[i])
                f = 250

                if class_name == 'vehicle':
                  color = (255,255,0)
                  original_height = 60
                elif class_name == 'pedestrian':
                  color = (0,255,255)
                  original_height = 63
                else:
                  color = (255,0,255)
                  original_height = 60
                dist = (original_height * f)/ (output_boxes[i][3] - output_boxes[i][1])
                dist = round(dist/12,5)
                if dist<=7.5:
                  color = (0,0,255)
                box=output_boxes[i]
                
                cv2.rectangle(image_np_with_detections, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
                cv2.rectangle(image_np_with_detections, (bbox[0], bbox[1]-30), (bbox[0]+(len(class_name))*17, bbox[1]), color, -1)
                cv2.putText(image_np_with_detections, class_name+"-"+str(round(dist,2))+"ft ",(bbox[0], bbox[1]-10),0, 0.75, (255,255,255),2)         
            
            for (object_id, centroid) in objects.items():
              text = "id{}".format(object_id)
              #print(centroid)
              cv2.putText(image_np_with_detections, text,(centroid[2], centroid[3]),0, 0.75, (255,255,255),2)         
              cv2.circle(image_np_with_detections, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

            currentframe += 1
            fps.update()
            out.write(image_np_with_detections)
            
        fps.stop()
        print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
        print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
        cap.release()
        out.release()

[INFO] elasped time: 384.32
[INFO] approx. FPS: 5.20
[INFO] elasped time: 369.89
[INFO] approx. FPS: 5.41
[INFO] elasped time: 369.52
[INFO] approx. FPS: 5.41
[INFO] elasped time: 366.78
[INFO] approx. FPS: 5.45
